In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, accuracy_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
# Câu 1
# Đọc dữ liệu
df = pd.read_excel("Dethi_data.xlsx", sheet_name="data")

# Tính thiếu dữ liệu theo cột
na_counts = df.isna().sum()

# Các nhóm cột cần kiểm tra
cols_basic = ["order","limit","delete","remove","clear","drop"]
group_cols = [f"{p}{i}" for p in ["c","d","f"] for i in range(1,7)]

# Kết quả kiểm tra cho các mệnh đề a..h
checks = {
    "a. order có 0 dữ liệu thiếu": na_counts["order"] == 0,
    "b. limit có 0 dữ liệu thiếu": na_counts["limit"] == 0,
    "c. delete có 0 dữ liệu thiếu": na_counts["delete"] == 0,
    "d. remove có 0 dữ liệu thiếu": na_counts["remove"] == 0,
    "e. Kích thước dữ liệu là (30000, 25)": df.shape == (30000, 25),
    "f. clear có 0 dữ liệu thiếu": na_counts["clear"] == 0,
    "g. tất cả ci, di, fi (i=1..6) có 0 dữ liệu thiếu": na_counts[group_cols].sum() == 0,
    "h. drop có 0 dữ liệu thiếu": na_counts["drop"] == 0,
}

# In tóm tắt
print("Kích thước:", df.shape)
print("Thiếu dữ liệu ở các cột cơ bản:", {c:int(na_counts[c]) for c in cols_basic})
print("Tổng thiếu ở nhóm c,d,f (1..6):", int(na_counts[group_cols].sum()))
print("\nKết luận từng mệnh đề:")
for k,v in checks.items():
    print(f"{k}: {v}")
print("-------------------------------------------------")


Kích thước: (30000, 25)
Thiếu dữ liệu ở các cột cơ bản: {'order': 0, 'limit': 0, 'delete': 0, 'remove': 0, 'clear': 0, 'drop': 0}
Tổng thiếu ở nhóm c,d,f (1..6): 0

Kết luận từng mệnh đề:
a. order có 0 dữ liệu thiếu: True
b. limit có 0 dữ liệu thiếu: True
c. delete có 0 dữ liệu thiếu: True
d. remove có 0 dữ liệu thiếu: True
e. Kích thước dữ liệu là (30000, 25): True
f. clear có 0 dữ liệu thiếu: True
g. tất cả ci, di, fi (i=1..6) có 0 dữ liệu thiếu: True
h. drop có 0 dữ liệu thiếu: True
-------------------------------------------------


In [2]:
# Câu 2
cols = ["order","limit","default","c1","d1","f1","c2","d2","f2","f3","f4","f5","f6"]
df2 = df[cols].dropna()
X = df2[["c1","d1","f1","c2","d2","f2","f3","f4","f5","f6"]]
y_reg = df2["limit"]
y_cls = df2["default"]

X_train, X_test, y_train, y_test = train_test_split(X, y_reg, test_size=0.1, random_state=10)

print("df_train:", X_train.shape)
print("df_test:", X_test.shape)
print("-------------------------------------------------")


df_train: (27000, 10)
df_test: (3000, 10)
-------------------------------------------------


In [3]:
# Câu 3
# Chọn các cột cần thiết
cols = ["order","limit","default","c1","d1","f1","c2","d2","f2","f3","f4","f5","f6"]
df2 = df[cols].dropna()

# Tạo X, y và cột order để chia train/test
X = df2[["c1","d1","f1","c2","d2","f2","f3","f4","f5","f6"]]
y_reg = df2["limit"]
y_cls = df2["default"]
order_vals = df2["order"]

# Chia dữ liệu (test 10%, random_state=10)
X_train, X_test, y_reg_train, y_reg_test, y_cls_train, y_cls_test, order_train, order_test = train_test_split(
    X, y_reg, y_cls, order_vals, test_size=0.1, random_state=10, shuffle=True
)

# In ra 5 giá trị đầu tiên
print("5 đầu df_train:", order_train.tolist()[:5])
print("5 đầu df_test:", order_test.tolist()[:5])
print("-------------------------------------------------")


5 đầu df_train: [2664, 1392, 18817, 485, 26435]
5 đầu df_test: [20413, 1297, 3907, 20455, 5201]
-------------------------------------------------


In [4]:
sets = {
    "a": [24449, 4368, 5750, 13544, 5330],
    "c": [28018, 17729, 29200, 7294, 17674],
    "e": [2664, 1392, 18817, 485, 26435],
    "g": [20413, 1297, 3907, 20455, 5201]
}

for key, idx_list in sets.items():
    result = []
    for i in idx_list:
        if i in order_train.values:
            result.append("train")
        elif i in order_test.values:
            result.append("test")
        else:
            result.append("not found")
    print(f"{key}: {result}")
print("-------------------------------------------------")


a: ['test', 'test', 'test', 'test', 'test']
c: ['train', 'train', 'train', 'train', 'train']
e: ['train', 'train', 'train', 'train', 'train']
g: ['test', 'test', 'test', 'test', 'test']
-------------------------------------------------


In [5]:
# Câu 4
cols = ["order","limit","default","c1","d1","f1","c2","d2","f2","f3","f4","f5","f6"]
df2 = df[cols].dropna()

X = df2[["c1","d1","f1","c2","d2","f2","f3","f4","f5","f6"]]
y = df2["limit"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=10)
model = LinearRegression().fit(X_train, y_train)
y_pred = model.predict(X_test)

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print("R2 =", r2)
print("MAE =", mae)
print("-------------------------------------------------")


R2 = 0.3003500023931771
MAE = 82906.8479776912
-------------------------------------------------


In [6]:
# Câu 5
sample = {"c1":-1.0,"d1":9640.0,"f1":15134.0,"c2":-2.0,"d2":7404.0,"f2":0.0,
          "f3":7002.0,"f4":8167.0,"f5":3996.0,"f6":2000.0}
print("Câu 5:", round(model.predict(pd.DataFrame([sample]))[0], 2))
print("-------------------------------------------------")


Câu 5: 213447.42
-------------------------------------------------


In [7]:
# Câu 6
use = df[["c1","d1","f1","drop"]].dropna()

# chia theo index để các mô hình khác dùng chung split
train_idx, test_idx = train_test_split(use.index, test_size=0.1, random_state=10, shuffle=True)
train = use.loc[train_idx]
# test = use.loc[test_idx]  # không cần cho C6

X_tr = train[["c1","d1","f1"]]
y_tr = train["drop"]

knn = KNeighborsClassifier(n_neighbors=20)
knn.fit(X_tr, y_tr)
acc_tr = accuracy_score(y_tr, knn.predict(X_tr))
print(acc_tr)
print("-------------------------------------------------")

0.5292962962962963
-------------------------------------------------


In [8]:
# Câu 7
use = df[["c1","d1","f1","drop"]].dropna()

train_idx, test_idx = train_test_split(use.index, test_size=0.1, random_state=10, shuffle=True)
train = use.loc[train_idx]

X_tr = train[["c1","d1","f1"]]
y_tr = train["drop"]

knn = KNeighborsClassifier(n_neighbors=20).fit(X_tr, y_tr)
knn_prob_train = knn.predict_proba(X_tr).max(axis=1)

print("Câu 7:", np.mean(knn_prob_train))           # 0.5374185185
print("Câu 7 (round 4):", round(np.mean(knn_prob_train), 2))  # 0.5374
print("-------------------------------------------------")


Câu 7: 0.5371074074074075
Câu 7 (round 4): 0.54
-------------------------------------------------


In [9]:
# Câu 8
# Chuẩn bị cột cần thiết
cols = ["c1","d1","f1","drop","c2","d2","f2","delete","f3","f4","f5","f6","limit"]
df2 = df[cols].dropna()

# Chia dữ liệu train/test giống giai đoạn 1
train_idx, test_idx = train_test_split(df2.index, test_size=0.1, random_state=10, shuffle=True)
train = df2.loc[train_idx].copy()
test  = df2.loc[test_idx].copy()

# KNN dự đoán nhãn "drop"
X_knn_train = train[["c1","d1","f1"]]
y_knn_train = train["drop"]
knn = KNeighborsClassifier(n_neighbors=20)
knn.fit(X_knn_train, y_knn_train)
knn_prob_train = knn.predict_proba(X_knn_train).max(axis=1)

# GNB dự đoán nhãn "delete"
X_gnb_train = train[["c2","d2","f2"]]
y_gnb_train = train["delete"]
gnb = GaussianNB()
gnb.fit(X_gnb_train, y_gnb_train)
gnb_prob_train = gnb.predict_proba(X_gnb_train).max(axis=1)

# Tạo tập đặc trưng hybrid trước Polynomial
X_hyb_train = pd.concat([
    pd.Series(knn_prob_train, name="knn_feature", index=train.index),
    pd.Series(gnb_prob_train, name="gnb_feature", index=train.index),
    train[["f3","f4","f5","f6"]]
], axis=1)

# Tính thống kê min, mean, var, max
arr = X_hyb_train.values.astype(float)
print("Min:", np.min(arr))
print("Mean:", np.mean(arr))
print("Var:", np.var(arr))
print("Max:", np.max(arr))
print("-------------------------------------------------")


Min: 0.0
Mean: 3372.6071102407514
Var: 194366041.855579
Max: 896040.0
-------------------------------------------------


In [10]:
# Câu 9
# 1) Đọc và chuẩn bị đúng các cột
cols = ["order","limit","c1","d1","f1","drop","c2","d2","f2","delete","f3","f4","f5","f6"]
data = df[cols].dropna()

# 2) Chia train/test MỘT lần dùng chung cho tất cả
train_idx, test_idx = train_test_split(data.index, test_size=0.1, random_state=10, shuffle=True)
train, test = data.loc[train_idx].copy(), data.loc[test_idx].copy()

# 3) KNN: <c1,d1,f1> -> drop (lấy max prob)
knn = KNeighborsClassifier(n_neighbors=20).fit(train[["c1","d1","f1"]], train["drop"])
knn_prob_tr = knn.predict_proba(train[["c1","d1","f1"]]).max(axis=1)
knn_prob_te = knn.predict_proba(test[["c1","d1","f1"]]).max(axis=1)

# 4) GNB: <c2,d2,f2> -> delete (lấy max prob)
gnb = GaussianNB().fit(train[["c2","d2","f2"]], train["delete"])
gnb_prob_tr = gnb.predict_proba(train[["c2","d2","f2"]]).max(axis=1)
gnb_prob_te = gnb.predict_proba(test[["c2","d2","f2"]]).max(axis=1)

# 5) Ma trận hybrid (train/test) trước Polynomial
X_hyb_tr = pd.DataFrame({
    "knn_feature": knn_prob_tr,
    "gnb_feature": gnb_prob_tr
}, index=train.index).join(train[["f3","f4","f5","f6"]])

X_hyb_te = pd.DataFrame({
    "knn_feature": knn_prob_te,
    "gnb_feature": gnb_prob_te
}, index=test.index).join(test[["f3","f4","f5","f6"]])

# 6) Polynomial(deg=2, include_bias=True) + LinearRegression
poly = PolynomialFeatures(degree=2, include_bias=True)
X_poly_tr = poly.fit_transform(X_hyb_tr)
X_poly_te = poly.transform(X_hyb_te)

lin = LinearRegression().fit(X_poly_tr, train["limit"])
y_pred_te = lin.predict(X_poly_te)

# 7) Chỉ số đánh giá
r2 = r2_score(test["limit"], y_pred_te)
mae = mean_absolute_error(test["limit"], y_pred_te)

# In ra với độ chính xác cao
print("R2 =", "{:.17f}".format(r2))   # 0.19126519272401998
print("MAE =", "{:.13f}".format(mae)) # 91801.6444067237


#=> MLR tốt hơn vì:

#Có R² cao hơn (dự báo sát thực tế hơn)

#Có MAE thấp hơn (sai số nhỏ hơn)

R2 = 0.19233141392717390
MAE = 91741.5131652632554


In [11]:
cols = ["order","limit","c1","d1","f1","drop","c2","d2","f2","delete","f3","f4","f5","f6"]
data = df[cols].dropna()

# ===== 1) Chia split MỘT lần, dùng chung cho mọi thứ =====
train_idx, test_idx = train_test_split(data.index, test_size=0.1, random_state=10, shuffle=True)
train, test = data.loc[train_idx], data.loc[test_idx]
print("Rows train/test:", len(train), len(test))        # -> 27000 / 3000

# ===== 2) Train KNN (drop) & GNB (delete) trên TRAIN =====
knn = KNeighborsClassifier(n_neighbors=20).fit(train[["c1","d1","f1"]], train["drop"])
gnb = GaussianNB().fit(train[["c2","d2","f2"]], train["delete"])

# ===== 3) Lấy prob tối đa trên TRAIN -> 2 feature =====
knn_prob_tr = knn.predict_proba(train[["c1","d1","f1"]]).max(axis=1)
gnb_prob_tr = gnb.predict_proba(train[["c2","d2","f2"]]).max(axis=1)

# ===== 4) GHÉP feature hybrid THEO INDEX TRAIN (tránh lệch hàng) =====
X_hyb_tr = pd.DataFrame({
    "knn_feature": pd.Series(knn_prob_tr, index=train.index),
    "gnb_feature": pd.Series(gnb_prob_tr, index=train.index),
})
# bắt buộc index bằng nhau
assert X_hyb_tr.index.equals(train.index)
X_hyb_tr = X_hyb_tr.join(train[["f3","f4","f5","f6"]])

# ===== 5) Polynomial(degree=2, include_bias=True) + LinearRegression =====
poly = PolynomialFeatures(degree=2, include_bias=True)
X_poly_tr = poly.fit_transform(X_hyb_tr)
lin = LinearRegression().fit(X_poly_tr, train["limit"])

# một vài check nhỏ
print("include_bias =", poly.include_bias)              # -> True
print("poly_features =", X_poly_tr.shape[1])           # -> 28 (6 features tạo 28 terms bậc 2 có bias)

# ===== 6) Dự đoán cho sample của đề =====
sample = {"c1":-1.0,"d1":9640.0,"f1":15134.0,
          "c2":-2.0,"d2":7404.0,"f2":0.0,
          "f3":7002.0,"f4":8167.0,"f5":3996.0,"f6":2000.0}

x_knn = np.array([[sample["c1"], sample["d1"], sample["f1"]]])
x_gnb = np.array([[sample["c2"], sample["d2"], sample["f2"]]])
knn_f = knn.predict_proba(x_knn)[0].max()
gnb_f = gnb.predict_proba(x_gnb)[0].max()
print("knn_feature(sample) =", knn_f)                  # -> ~0.65
print("gnb_feature(sample) =", gnb_f)                  # -> ~0.6049781764

x_hyb = np.array([[knn_f, gnb_f, sample["f3"], sample["f4"], sample["f5"], sample["f6"]]])
x_poly = poly.transform(x_hyb)
y_pred = lin.predict(x_poly)[0]
print("Câu 10 (Pipeline B) =", round(y_pred, 2))       # -> ~164627.55
print("-------------------------------------------------")


Rows train/test: 27000 3000
include_bias = True
poly_features = 28
knn_feature(sample) = 0.65
gnb_feature(sample) = 0.6049781763887933
Câu 10 (Pipeline B) = 163225.11
-------------------------------------------------


c:\Users\Hoang Nghi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\Users\Hoang Nghi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(
c:\Users\Hoang Nghi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(
